# BMI Predictor 🧮
---
## Training

In [1]:
# Libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib # save and load models

# Configuration: Random seed for reproducibility
RANDOM_STATE = 42

In [2]:
# 1. Load the cleaned BMI dataset
df_bmi = pd.read_csv('../data/bmi_cleaned.csv')

In [3]:
# 2. Define Features (X) and Target Variable (y)
X = df_bmi.drop(columns=['Index']) 
y = df_bmi['Index']

In [4]:
# 3. Split the data into Training (80%) and Testing (20%)
# We use stratify=y to ensure that the class distribution of 'Index' is the same
# in the training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [5]:
print(f"Size of X_train: {X_train.shape}")
print(f"Size of X_test: {X_test.shape}")

Size of X_train: (388, 4)
Size of X_test: (98, 4)


In [6]:
# (Preprocessing + Model) ⚙️

# 1. Define numeric features for scaling
# Exclude 'Gender_Encoded' as it is a binary variable.
numeric_features = ['Height', 'Weight', 'BMI_Value']

In [7]:
# 2. Create ColumnTransformer for preprocessing
# This will scale only the numeric features.
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_features)
    ],
    remainder='passthrough' # Keep other columns (Gender_Encoded) unchanged
)

In [8]:
# 3. Create Pipeline with Preprocessor and Random Forest Classifier
# We define the Pipeline, which sequentially applies the preprocessor and then the classifier.
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # We set up an initial Random Forest model for the baseline
    ('classifier', RandomForestClassifier(random_state=RANDOM_STATE)) 
])

In [9]:
# Train the model 🎯

print("Starting training...")

# 1. Training: The Pipeline applies scaling (fit_transform) and then trains the model.
rf_pipeline.fit(X_train, y_train)

# 2. Prediction on the test set
y_pred_base = rf_pipeline.predict(X_test)

# 3. Initial Evaluation
print("\n--- Results ---")
print(f"Accuracy on Test: {accuracy_score(y_test, y_pred_base):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_base))

Starting training...

--- Results ---
Accuracy on Test: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        26
           5       1.00      1.00      1.00        39

    accuracy                           1.00        98
   macro avg       1.00      1.00      1.00        98
weighted avg       1.00      1.00      1.00        98



In [10]:
# Asignamos el pipeline entrenado al nombre 'best_model' para seguir la convención
best_model = rf_pipeline 
final_accuracy = 1.0000 # La precisión que obtuviste

# 1. Definir el nombre del archivo de guardado
model_filename = f'best_rf_pipeline_acc{final_accuracy:.4f}.joblib'

# 2. Guardar el Pipeline completo
joblib.dump(best_model, model_filename)

print(f"\n✅ Pipeline saved: {model_filename}")
print("Training is complete and the model is ready for deployment.")


✅ Pipeline saved: best_rf_pipeline_acc1.0000.joblib
Training is complete and the model is ready for deployment.
